In [16]:
from pynq import Overlay
from pynq import MMIO
import time



In [17]:
# Load bitstream
ol = Overlay('SBUS_DEMO.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')



  
1.1873517036437988 seconds to program bitstream


In [27]:
# definitions
IP_BASE_ADDRESS = 0X43C00000
ADDRESS_RANGE = 0x10000

RX_BUF_REG       =   0x1000        #Receiver Buffer Register              LCR[7] = 0
TX_HOLD_REG      =   0x1000        #Transmitter Holding Register          LCR[7] = 0
INT_EN_REG       =   0x1004        #Interrupt Enable Register             LCR[7] = 0
INT_ID_REG       =   0x1008        #Interrupt Identification Register     LCR[7] = x
FIFO_CNTRL_REG   =   0x1008        #FIFO Control Register                 LCR[7] = x (write), LCR[7] = 1 (read)
LINE_CNTRL_REG   =   0x100C        #Line Control Register                 LCR[7] = x
MODEM_CNTRL_REG  =   0x1010        #Modem Control Register                LCR[7] = x
LINE_STATUS_REG  =   0x1014        #Line Status Register                  LCR[7] = x
MODEM_STATUS_REG =   0x1018        #Modem Status Register                 LCR[7] = x
SCRATCH_REG      =   0x101C        #Scratch Register                      LCR[7] = x
DIV_LATCH_LSB    =   0x1000        #Divisor Latch LSB Register            LCR[7] = 1
DIV_LATCH_MSB    =   0x1004        #Divisor Latch MSB Register            LCR[7] = 1

NUM_BYTES        =   25
START_PACKET     =   0x0F
STOP_PACKET      =   0x00



TEST_ADDRESS = 0x101C

# testing
mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

data = 0xDEADBEEF
data2 = 0xDE

#mmio.write(ADDRESS_OFFSET, data)
#result = mmio.read(ADDRESS_OFFSET)

mmio.write(TEST_ADDRESS, data2)

result = mmio.read(0x101C)

print(data2)
print(result)


222
222


In [73]:
# Intro
print('----------------------------------------------------------')
print('         Can we make the UART work....                    ')
print('----------------------------------------------------------')


# TODO

# configure UART core
# read from eternal pin @ 100k BAUD

# configureing UART core

# enable divisor latch, set baud divisor
mmio.write(LINE_CNTRL_REG, 0x80)
mmio.write(DIV_LATCH_LSB, 0x3E) # divide by 62
mmio.write(DIV_LATCH_MSB, 0x0)

# configure data exchange, 100k BAUD, even parity, 2 stop bits, 8 data bits
mmio.write(LINE_CNTRL_REG, 0x1F)

# enable and configure FIFO
mmio.write(FIFO_CNTRL_REG, 0x1)

# reset FIFOs
mmio.write(FIFO_CNTRL_REG, 0x7)
mmio.write(FIFO_CNTRL_REG, 0x1)


print('YES!')



----------------------------------------------------------
         Can we make the UART work....                    
----------------------------------------------------------
YES!


In [72]:
# grabbing data from the UART bus

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES-1)
BYTES_READ = 0



while(BYTES_READ != (NUM_BYTES - 1)):
    
    #read from the RX FIFO
    while((mmio.read(LINE_STATUS_REG) & BIT_ONE) != 1):
        pass
    
    #print(mmio.read(RX_BUF_REG))
    #print(idx)
    
    #if(mmio.read(RX_BUF_REG) != 240):
        #pass
        #print('hard pass')
    temp = mmio.read(RX_BUF_REG)
    if (temp != 240):
        print(temp)
    else:
        buffer[BYTES_READ] = mmio.read(RX_BUF_REG)
        BYTES_READ += 1

        
    
print(''.join('%02x'%i for i in buffer))
    



    

94
122
70
19
244
130
117
206
127
127
127
127
127
127
0
94
94
202
70
17
244
194
117
204
127
127
127
127
127
127
0
94
202
70
17
244
192
117
204
127
127
127
127
127
127
0
94
202
70
17
244
194
117
204
127
127
127
127
127
127
0
92
202
70
17
244
194
117
204
127
127
127
63
127
127
0
94
122
70
17
244
192
117
206
127
127
127
127
127
127
0
92
202
70
17
244
194
117
204
127
127
127
127
127
127
0
94
122
70
19
244
192
117
206
127
127
127
127
127
127
0
94
122
70
17
244
194
117
204
127
127
127
127
127
127
0
92
202
70
17
244
194
117
204
63
127
127
127
127
127
0
94
202
70
19
244
192
117
206
127
127
127
127
127
127
0
92
202
70
17
244
194
117
204
127
127
127
127
127
127
0
94
122
70
17
244
192
117
206
127
127
127
127
127
63
0
94
202
70
19
244
194
117
204
127
127
127
127
127
127
0
92
202
70
17
244
194
117
204
127
127
127
127
127
127
0
94
202
70
19
244
130
117
206
127
127
127
127
127
127
0
92
58
70
17
244
194
117
204
127
63
127
127
127
127
0
94
122
70
17
244
192
117
206
127
127
127
127
127
127
0
94
122
70
17

222
14
185
196
245
192
117
206
127
127
127
127
127
127
0
222
221
203
120
234
9
234
204
127
127
127
127
127
127
0
220
93
84
250
209
9
234
204
127
127
127
63
127
127
0
222
189
68
78
209
9
234
206
127
127
127
127
127
127
0
220
125
36
20
209
9
234
204
127
127
127
127
127
127
0
222
221
87
47
245
192
117
204
127
127
127
127
127
63
0
222
253
175
76
33
9
234
204
127
127
127
127
127
127
0
220
108
26
215
245
194
117
204
127
127
63
127
127
127
0
94
28
152
215
245
194
117
204
127
127
127
127
127
127
0
222
202
97
112
33
9
234
204
127
127
127
127
127
127
0
222
93
205
78
97
9
234
206
127
127
127
127
127
127
0
94
5
243
161
244
194
222
73
18
55
244
192
117
206
127
127
127
127
127
127
0
222
121
182
139
244
194
117
204
127
127
127
127
127
127
0
92
54
229
55
244
194
117
204
127
127
127
127
127
127
0
94
134
231
117
244
192
117
206
127
127
127
127
127
127
0
94
70
37
119
244
194
117
204
127
127
127
127
127
127
0
94
118
68
117
244
192
117
206
127
127
127
127
127
127
0
94
68
45
70
244
194
117
204
127
127
127
1

127
127
127
127
0
220
107
109
206
245
194
117
204
63
127
127
127
127
127
0
92
146
227
91
244
194
117
204
127
127
127
127
127
127
0
94
210
225
17
245
194
117
204
127
127
127
127
127
127
0
92
105
109
79
245
194
117
204
127
127
127
127
127
127
0
94
196
107
0
244
130
117
206
127
127
127
127
127
127
0
222
70
161
15
244
194
117
204
127
127
127
127
127
127
0
92
5
241
170
244
194
117
204
127
127
127
127
127
127
0
94
112
177
170
244
194
117
204
127
127
127
127
127
127
0
94
8
227
119
244
194
117
204
127
127
127
127
127
127
0
94
106
185
221
244
192
117
206
127
127
127
127
127
127
0
94
96
217
221
222
8
166
19
244
192
117
206
127
127
127
127
127
127
0
222
8
166
19
244
194
117
206
127
127
127
127
127
127
0
220
204
76
17
244
194
117
204
127
127
127
127
127
127
0
222
184
166
19
244
130
117
206
127
127
127
127
127
127
0
220
78
83
17
244
194
117
204
63
127
127
127
127
127
0
94
184
166
19
244
192
117
206
127
127
127
127
127
127
0
94
14
83
17
244
194
117
204
127
127
127
127
127
127
0
220
212
76
17
244
194

KeyboardInterrupt: 

In [ ]:
# TRAVIS IDEAS for infinite graph display

while !quit:
  get packet
  update the graph
  send stuff if the user wants to send stuff
    i.e. break commands